### fetch and save data

In [1]:
import random
import pickle
import nltk
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\0\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [2]:
reviews = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]
print(len(reviews))

2000


In [3]:
print(reviews[4])

(['synopsis', ':', 'a', 'mentally', 'unstable', 'man', 'undergoing', 'psychotherapy', 'saves', 'a', 'boy', 'from', 'a', 'potentially', 'fatal', 'accident', 'and', 'then', 'falls', 'in', 'love', 'with', 'the', 'boy', "'", 's', 'mother', ',', 'a', 'fledgling', 'restauranteur', '.', 'unsuccessfully', 'attempting', 'to', 'gain', 'the', 'woman', "'", 's', 'favor', ',', 'he', 'takes', 'pictures', 'of', 'her', 'and', 'kills', 'a', 'number', 'of', 'people', 'in', 'his', 'way', '.', 'comments', ':', 'stalked', 'is', 'yet', 'another', 'in', 'a', 'seemingly', 'endless', 'string', 'of', 'spurned', '-', 'psychos', '-', 'getting', '-', 'their', '-', 'revenge', 'type', 'movies', 'which', 'are', 'a', 'stable', 'category', 'in', 'the', '1990s', 'film', 'industry', ',', 'both', 'theatrical', 'and', 'direct', '-', 'to', '-', 'video', '.', 'their', 'proliferation', 'may', 'be', 'due', 'in', 'part', 'to', 'the', 'fact', 'that', 'they', "'", 're', 'typically', 'inexpensive', 'to', 'produce', '(', 'no', 'spe

In [4]:
random.shuffle(reviews)

In [5]:
stop_words = stopwords.words("english") + ['\'', '-', '(', ')', ':', '"', ';', '--']

train = './fasttext-train.txt'
f = open(train, 'w')

pos_test = []
neg_test = []

nc, pc = 0, 0
for text, tag in reviews:
#     text = ' '.join([w.lower() for w in text if not w.lower() in stop_words])
    text = ' '.join(text)
    if tag == 'pos':
        pc += 1
        if pc <= 850:
            f.write("__label__{} {}\n".format(tag, text))
        else:
            pos_test.append(text)
    else:
        nc += 1
        if nc <= 850:
            f.write("__label__{} {}\n".format(tag, text))
        else:
            neg_test.append(text)
f.close()

with open("fasttext-test.pkl", 'wb') as f:
    pickle.dump(pos_test, f)
    pickle.dump(neg_test, f)

### fastText text classification

In [6]:
import pickle
import fasttext

In [7]:
model = fasttext.train_supervised(train, label_prefix='__label__', lr=1.0, minCount=2, dim=100, epoch=50, verbose=True)
model.save_model("model_train.bin")

In [8]:
model.predict('this is good')

(('__label__neg',), array([0.83758378]))

In [9]:
def sentiscore(text):
    return model.predict(text)[0][0].rsplit('__', 1)[1]

In [10]:
with open("fasttext-test.pkl", 'rb') as f:
    pos_test = pickle.load(f)
    neg_test = pickle.load(f)

In [11]:
neg_right, pos_right = 0, 0
for t in neg_test:
    senti = sentiscore(t)
    if senti == 'neg':
        neg_right += 1
for t in pos_test:
    senti = sentiscore(t)
    if senti == 'pos':
        pos_right += 1
print('pos right', pos_right / len(pos_test))
print('neg right', neg_right / len(neg_test))
print('total', (pos_right + neg_right) / (len(pos_test) + len(neg_test)))

pos right 0.86
neg right 0.8133333333333334
total 0.8366666666666667
